<a href="https://colab.research.google.com/github/Teratoma111/amazon_review_analysis/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import numpy as np  
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(include_lengths = True,
                  preprocessing = generate_bigrams)

LABEL = data.LabelField()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random

fields = {'reviewText': ('text', TEXT), 'overall': ('label', LABEL)}
train_data= data.TabularDataset.splits(
                                        path = '/content/drive/MyDrive',
                                        train = "Musical_Instruments_5.json",
                                        format = 'json',
                                        fields = fields)[0]
for i in range(0, len(train_data)):
  if(len(train_data[i].text) < 1):
    train_data[i].text = train_data[3].text
    train_data[i].label = train_data[3].label

In [ ]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))
test_data = valid_data

In [ ]:


TEXT.build_vocab(train_data, 
                 max_size = 30000,
                 vectors = "fasttext.simple.300d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    #sort_within_batch = False,
    sort_within_batch = True, 
    device = device)

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        embedded = self.dropout(self.embedding(text))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.65
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:
def get_weights(iterator):
  weights = torch.zeros(len(LABEL.vocab))
  for batch in iterator:
    for i in batch.label:
      weights[i.cpu().numpy()] +=1
  for i in range(0, len(LABEL.vocab)):
    weights[i] = (1 / weights[i]) * 100
  return weights

In [ ]:
print(get_weights(train_iterator))

tensor([0.0205, 0.0692, 0.1838, 0.5882, 0.6993])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss(weight =  get_weights(train_iterator))

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from sklearn.metrics import f1_score
def categorical_accuracy(preds, y):
    top_pred = preds.argmax(1, keepdim = True)
    y_score = top_pred.detach().cpu().numpy()
    y_true = y.view_as(top_pred).cpu().numpy()
    rec = f1_score(y_true, y_score, average = 'weighted')
    return rec

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 60

best_valid_score = float('0')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_score = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_score = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    if valid_score > best_valid_score:
        best_valid_score= valid_score
        torch.save(model.state_dict(), 'model.pt')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1 score {valid_score*100:.2f}%')
    print(f'\tTrain Loss: {train_loss:.3f} | Train F1 score: {train_score*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 11s
	 Val. Loss: 1.608 |  Val. F1 score 44.68%
	Train Loss: 1.618 | Train F1 score: 42.24%
Epoch: 02 | Epoch Time: 0m 11s
	Train Loss: 1.588 | Train F1 score: 39.65%
Epoch: 03 | Epoch Time: 0m 11s
	 Val. Loss: 1.653 |  Val. F1 score 48.15%
	Train Loss: 1.554 | Train F1 score: 33.60%
Epoch: 04 | Epoch Time: 0m 11s
	Train Loss: 1.509 | Train F1 score: 38.48%
Epoch: 05 | Epoch Time: 0m 11s
	Train Loss: 1.449 | Train F1 score: 39.96%
Epoch: 06 | Epoch Time: 0m 11s
	Train Loss: 1.357 | Train F1 score: 36.24%
Epoch: 07 | Epoch Time: 0m 11s
	Train Loss: 1.326 | Train F1 score: 39.97%
Epoch: 08 | Epoch Time: 0m 11s
	Train Loss: 1.220 | Train F1 score: 38.88%
Epoch: 09 | Epoch Time: 0m 11s
	Train Loss: 1.163 | Train F1 score: 41.80%
Epoch: 10 | Epoch Time: 0m 11s
	Train Loss: 1.126 | Train F1 score: 44.01%
Epoch: 11 | Epoch Time: 0m 11s
	Train Loss: 1.051 | Train F1 score: 42.84%
Epoch: 12 | Epoch Time: 0m 11s
	 Val. Loss: 2.601 |  Val. F1 score 51.50%
	Train Loss: 0.

In [ ]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_score = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test F1 score: {test_score*100:.2f}%')

Test Loss: 5.141 | Test F1 score: 60.15%
